In [3]:
rm -rf 

In [4]:
import sys
print(sys.executable)

/root/miniconda3/bin/python


In [5]:
import scimilarity
from scimilarity.utils import lognorm_counts, align_dataset
from scimilarity import CellQuery

In [6]:
from itertools import combinations
import numpy as np
import random
import torch
from typing import List, Union, Optional

from scimilarity.ontologies import (
    import_cell_ontology,
    get_id_mapper,
    get_all_ancestors,
    get_all_descendants,
    get_parents,
)

In [7]:
import numpy as np

In [8]:
# !pip install numpy==1.26.0

In [9]:
np.__version__

'1.26.0'

In [10]:
import torch
from datetime import datetime
import json
import os
import pandas as pd
import pytorch_lightning as pl
import torch
from torch import nn
import torch.nn.functional as F
from typing import Optional, List

from scimilarity.triplet_selector import TripletSelector
from scimilarity.nn_models import Encoder, Decoder

In [11]:
import scanpy as sc
import anndata as ad

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
from torch.utils.data import TensorDataset

In [14]:
# rm -r /root/autodl-tmp/train_path_A_S/Study_A/Sample_A_zarr

In [15]:
# rm -r /root/autodl-tmp/train_path_A_S/Study_S/Sample_S_zarr

In [16]:
import zarr

In [17]:
# !unzip /root/autodl-tmp/A_new_zarr.zip -d /root/autodl-tmp/train_path_A_S/Study_A

In [18]:
# !unzip /root/autodl-tmp/S_new_zarr.zip -d /root/autodl-tmp/train_path_A_S/Study_S

In [19]:
from collections import Counter
import numpy as np
import os
import pandas as pd
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from tqdm import tqdm
from typing import Optional

from scimilarity.zarr_dataset import ZarrDataset

In [20]:
class scDataset(Dataset):
    """A class that represent a collection of single cell datasets in zarr format."""

    def __init__(self, data_list, obs_celltype="cell_type", obs_study="orig.ident"):
        """Constructor.

        Parameters
        ----------
        data_list: list
            List of single-cell datasets.
        obs_celltype: str, default: "celltype_name"
            Cell type name.
        obs_study: str, default: "study"
            Study name.
        """

        self.data_list = data_list
        self.ncells_list = [data.shape[0] for data in data_list]
        self.ncells = sum(self.ncells_list)
        self.obs_celltype = obs_celltype
        self.obs_study = obs_study

        self.data_idx = [
            n for n in range(len(self.ncells_list)) for i in range(self.ncells_list[n])
        ]
        self.cell_idx = [
            i for n in range(len(self.ncells_list)) for i in range(self.ncells_list[n])
        ]

    def __len__(self):
        return self.ncells

    def __getitem__(self, idx):
        # data, label, study
        data_idx = self.data_idx[idx]
        cell_idx = self.cell_idx[idx]
        return (
            self.data_list[data_idx].get_cell(cell_idx).toarray(),
            self.data_list[data_idx].get_obs(self.obs_celltype)[cell_idx],
            self.data_list[data_idx].get_obs(self.obs_study)[cell_idx],
        )


In [21]:
import zarr
# 假设我们有两个单细胞数据集，分别加载
A = ad.read_zarr("/root/autodl-tmp/train_path_A_S/Study_A/Sample_A_zarr")
S = ad.read_zarr("/root/autodl-tmp/train_path_A_S/Study_S/Sample_S_zarr")

In [22]:
A

AnnData object with n_obs × n_vars = 61000 × 34667
    obs: 'Capbatch', 'DMGM_DL', 'DMGM_UL', 'DMWM', 'NBB_case', 'PMI', 'RIN', 'RNA_snn_res.0.3', 'SCT_snn_res.0.1', 'Seqbatch', 'UMIs', 'age', 'cell_type', 'diagnosis', 'genes', 'mito_perc', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'orig.ident', 'pathology', 'percent.mt', 'pseudotime', 'region', 'ribo_perc', 'sample', 'seurat_cluster', 'seurat_clusters', 'sex', 'stage'
    layers: 'counts'

In [23]:
A.obs['cell_type'].value_counts()

cell_type
oligodendrocyte                   45697
astrocyte                          8211
neuron                             2803
oligodendrocyte precursor cell     2175
endothelial cell                   1708
lymphocyte                          406
Name: count, dtype: int64

In [24]:
# A.obs['cell_type'] = A.obs['cell_type'].replace({'Oligo1':'oligodendrocyte','opc':'oligodendrocyte precursor cell','neurons':'neuron','astrocytes':'astrocyte','vascular_cells': 'endothelial cell','lymphocytes': 'lymphocyte'})
# A.obs['cell_type'].value_counts()

In [25]:
# A = A[~A.obs['cell_type'].isin(['immune']), :]
# A.obs['cell_type'].value_counts()

In [26]:
# A.shape[0]

In [27]:
S

AnnData object with n_obs × n_vars = 45215 × 34667
    obs: 'Capbatch', 'DMGM_DL', 'DMGM_UL', 'DMWM', 'NBB_case', 'PMI', 'RIN', 'RNA_snn_res.0.3', 'SCT_snn_res.0.1', 'Seqbatch', 'UMIs', 'age', 'cell_type', 'diagnosis', 'genes', 'mito_perc', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'orig.ident', 'pathology', 'percent.mt', 'pseudotime', 'region', 'ribo_perc', 'sample', 'seurat_cluster', 'seurat_clusters', 'sex', 'stage'
    layers: 'counts'

In [28]:
S.obs['cell_type'].value_counts()

cell_type
neuron                            21539
oligodendrocyte                   12394
astrocyte                          5381
oligodendrocyte precursor cell     3844
microglial cell                    1425
endothelial cell                    374
phagocyte                           258
Name: count, dtype: int64

In [29]:
# S.obs['cell_type'] = S.obs['cell_type'].replace({'OL-A':'oligodendrocyte','OL-B':'oligodendrocyte','OL-C':'oligodendrocyte','OPC':'oligodendrocyte precursor cell','IN-VIP':'neuron','EN-L4':'neuron','EN-L2-3-A':'neuron','EN-L2-3-B':'neuron','EN-L5-6':'neuron','EN-MIX':'neuron','EN-PYR':'neuron','IN-PVALB':'neuron','IN-SST':'neuron','IN-SV2C':'neuron','Astrocytes':'astrocyte','Endo cells': 'endothelial cell','Phagocytes': 'phagocyte','Microglia': 'micrglial cell'})
# S.obs['cell_type'].value_counts()

In [30]:
# S = S[~S.obs['cell_type'].isin(['Stromal cells']), :]
# S = S[~S.obs['cell_type'].isin(['T cells']), :]
# S = S[~S.obs['cell_type'].isin(['B cells']), :]
# S = S[~S.obs['cell_type'].isin(['Glia-MIX']), :]
# S.obs['cell_type'].value_counts()

In [31]:
# 将这两个数据集传递给 scDataset
data_list = [A, S]
sc_dataset = scDataset(data_list)

In [32]:
sc_dataset.ncells_list

[61000, 45215]

In [33]:
class MetricLearningDataModule(pl.LightningDataModule):
    """A class to encapsulate a collection of zarr datasets to train the model."""

    def __init__(
        self,
        train_path: str,
        gene_order: str,
        val_path: Optional[str] = None,
        obs_field: str = "celltype_name",
        batch_size: int = 1000,
        num_workers: int = 1,
    ):
        """Constructor.

        Parameters
        ----------
        train_path: str
            Path to folder containing all training datasets.
            All datasets should be in zarr format, aligned to a known gene space, and
            cleaned to only contain valid cell ontology terms.
        gene_order: str
            Use a given gene order as described in the specified file. One gene
            symbol per line.
            IMPORTANT: the zarr datasets should already be in this gene order
            after preprocessing.
        val_path: str, optional, default: None
            Path to folder containing all validation datasets.
        obs_field: str, default: "celltype_name"
            The obs key name containing celltype labels.
        batch_size: int, default: 1000
            Batch size.
        num_workers: int, default: 1
            The number of worker threads for dataloaders

        Examples
        --------
        >>> datamodule = MetricLearningZarrDataModule(
                batch_size=1000,
                num_workers=1,
                obs_field="celltype_name",
                train_path="train",
                gene_order="gene_order.tsv",
            )
        """

        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.batch_size = batch_size
        self.num_workers = num_workers

        # gene space needs be aligned to the given gene order
        with open(gene_order, "r") as fh:
            self.gene_order = [line.strip() for line in fh]

        self.n_genes = len(self.gene_order)  # used when creating training model

        train_data_list = []
        self.train_Y = []  # text labels
        self.train_study = []  # text studies

        if self.train_path[-1] != os.sep:
            self.train_path += os.sep

        self.train_file_list = [
    (
        root.replace(self.train_path, "").split(os.sep)[0],  # 获取根目录名
        dir_name.replace("_zarr", "")  # 去掉'_zarr'后缀
    )
    for root, dirs, files in os.walk(self.train_path)
    for dir_name in dirs  # 遍历所有子目录
    if dir_name.endswith("_zarr")  # 检查是否以'_zarr'结尾
]

        for study, sample in tqdm(self.train_file_list):
            data_path = os.path.join(
                self.train_path, study, sample + "_zarr"
            )
            if os.path.isdir(data_path):
                zarr_data = ZarrDataset(data_path)
                train_data_list.append(zarr_data)
                self.train_Y.extend(zarr_data.get_obs(obs_field).astype(str).tolist())
                self.train_study.extend(zarr_data.get_obs("orig.ident").astype(str).tolist())

        # Lazy load training data from list of zarr datasets
        self.train_dataset = scDataset(train_data_list)

        self.class_names = set(self.train_Y)
        self.label2int = {label: i for i, label in enumerate(self.class_names)}
        self.int2label = {value: key for key, value in self.label2int.items()}

        self.val_dataset = None
        if self.val_path is not None:
            val_data_list = []
            self.val_Y = []
            self.val_study = []

            if self.val_path[-1] != os.sep:
                self.val_path += os.sep

            self.val_file_list = [
                (
                    root.replace(self.val_path, "").split(os.sep)[0],
                    dirs[0].replace("zarr", ""),
                )
                for root, dirs, files in os.walk(self.val_path)
                if dirs and dirs[0].endswith("zarr")
            ]

            for study, sample in tqdm(self.val_file_list):
                data_path = os.path.join(
                    self.val_path, study, sample, sample + "zarr"
                )
                if os.path.isdir(data_path):
                    zarr_data = ZarrDataset(data_path)
                    val_data_list.append(zarr_data)
                    self.val_Y.extend(zarr_data.get_obs(obs_field).astype(str).tolist())
                    self.val_study.extend(
                        zarr_data.get_obs("study").astype(str).tolist()
                    )

            # Lazy load val data from list of zarr datasets
            self.val_dataset = scDataset(val_data_list)

    def get_sampler_weights(
        self, labels: list, studies: Optional[list] = None
    ) -> WeightedRandomSampler:
        """Get weighted random sampler.

        Parameters
        ----------
        dataset: scDataset
            Single cell dataset.

        Returns
        -------
        WeightedRandomSampler
            A WeightedRandomSampler object.
        """

        if studies is None:
            class_sample_count = Counter(labels)
            sample_weights = torch.Tensor([1.0 / class_sample_count[t] for t in labels])
        else:
            class_sample_count = Counter(labels)
            study_sample_count = Counter(studies)
            class_sample_count = {
                x: np.log1p(class_sample_count[x] / 1e4) for x in class_sample_count
            }
            study_sample_count = {
                x: np.log1p(study_sample_count[x] / 1e5) for x in study_sample_count
            }
            sample_weights = torch.Tensor(
                [
                    1.0 / class_sample_count[labels[i]] / study_sample_count[studies[i]]
                    for i in range(len(labels))
                ]
            )
        return WeightedRandomSampler(sample_weights, len(sample_weights))


    def collate(self, batch):
        """Collate tensors.

        Parameters
        ----------
        batch:
            Batch to collate.

        Returns
        -------
        tuple
            A Tuple[torch.Tensor, torch.Tensor, list] containing information
            on the collated tensors.
        """

        profiles, labels, studies = tuple(
            map(list, zip(*batch))
        )  # tuple([list(t) for t in zip(*batch)])
        return (
            torch.squeeze(torch.Tensor(np.vstack(profiles))),
            torch.Tensor([self.label2int[l] for l in labels]),  # text to int labels
            studies,
        )


    def train_dataloader(self) -> DataLoader:
        """Load the training dataset.

        Returns
        -------
        DataLoader
            A DataLoader object containing the training dataset.
        """

        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            drop_last=True,
            sampler=self.get_sampler_weights(self.train_Y, self.train_study),
            collate_fn=self.collate,
        )


    def val_dataloader(self) -> DataLoader:
        """Load the validation dataset.

        Returns
        -------
        DataLoader
            A DataLoader object containing the validation dataset.
        """

        if self.val_dataset is None:
            return None
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            drop_last=True,
            sampler=self.get_sampler_weights(self.val_Y, self.val_study),
            collate_fn=self.collate,
        )


    def test_dataloader(self) -> DataLoader:
        """Load the test dataset.

        Returns
        -------
        DataLoader
            A DataLoader object containing the test dataset.
        """

        return self.val_dataloader()
    
    def __getitem__(self, idx):
        # 获取稀疏矩阵数据
        csr_matrix_data = self.data_list[idx].get_cell(cell_idx)

        # 将稀疏矩阵转为密集矩阵
        dense_array = csr_matrix_data.toarray()  # 替换 .A 为 .toarray()

        # 返回转为密集矩阵的结果
        return dense_array

In [34]:
# 准备数据和文件
train_path = "/root/autodl-tmp/train_path_A_S/"
# Val_path = "/root/autodl-tmp/Val_path/S_new_zarr"
gene_order_file = "/root/autodl-tmp/genes.tsv"

# 创建数据模块实例
datamodule = MetricLearningDataModule(
    train_path=train_path,          # 第一个数据集作为训练集
    val_path=None,            # 第二个数据集作为验证集
    gene_order=gene_order_file,
    obs_field="cell_type",         # 细胞类型标签字段
    batch_size=1000,
    num_workers=1
)

100%|██████████| 2/2 [00:00<00:00, 44.09it/s]


In [35]:
int2label = datamodule.int2label
int2label

{0: 'oligodendrocyte',
 1: 'lymphocyte',
 2: 'oligodendrocyte precursor cell',
 3: 'phagocyte',
 4: 'endothelial cell',
 5: 'microglial cell',
 6: 'neuron',
 7: 'astrocyte'}

In [89]:
df = pd.DataFrame(list(int2label.items()))

In [90]:
# 保存为 CSV 文件
df.to_csv("label_ints.csv", index=False)

In [36]:
label2int = datamodule.label2int
label2int

{'oligodendrocyte': 0,
 'lymphocyte': 1,
 'oligodendrocyte precursor cell': 2,
 'phagocyte': 3,
 'endothelial cell': 4,
 'microglial cell': 5,
 'neuron': 6,
 'astrocyte': 7}

In [37]:
datamodule.class_names

{'astrocyte',
 'endothelial cell',
 'lymphocyte',
 'microglial cell',
 'neuron',
 'oligodendrocyte',
 'oligodendrocyte precursor cell',
 'phagocyte'}

In [38]:
int2label.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [39]:
labels = np.array([0, 1, 2, 3, 4, 5, 6, 7])
labels

array([0, 1, 2, 3, 4, 5, 6, 7])

In [40]:
len(datamodule.train_Y)

106215

In [41]:
datamodule.train_file_list

[('Study_A', 'Sample_A'), ('Study_S', 'Sample_S')]

In [42]:
train_dataloader = datamodule.train_dataloader()

In [43]:
train_dataloader

In [44]:
print(train_dataloader.dataset)

In [45]:
for batch in train_dataloader:
    print(batch)  # 打印批次内容
    break 

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([1., 3., 6., 2., 1., 6., 1., 5., 6., 1., 3., 1., 7., 2., 6., 1., 0., 6.,
        4., 5., 7., 1., 6., 6., 0., 6., 1., 4., 3., 5., 0., 6., 6., 7., 7., 5.,
        4., 3., 6., 5., 2., 0., 3., 0., 0., 3., 1., 1., 1., 0., 5., 1., 0., 6.,
        0., 0., 0., 0., 1., 0., 2., 6., 0., 0., 0., 6., 1., 6., 2., 0., 0., 7.,
        6., 6., 4., 7., 1., 0., 2., 0., 7., 7., 7., 0., 7., 6., 0., 7., 6., 0.,
        0., 2., 1., 3., 5., 6., 2., 1., 1., 7., 6., 0., 2., 2., 0., 7., 4., 3.,
        7., 7., 5., 0., 7., 6., 2., 4., 1., 0., 4., 0., 6., 3., 4., 0., 4., 5.,
        5., 7., 6., 7., 3., 7., 6., 6., 6., 0., 0., 7., 6., 0., 6., 6., 1., 7.,
        4., 5., 3., 0., 7., 3., 0., 0., 0., 0., 6., 6., 2., 6., 4., 3., 0., 2.,
        1., 3., 5., 0., 

In [46]:
batch[0].shape

torch.Size([1000, 34667])

In [47]:
class TripletSelector:
    """For each anchor-positive pair, mine negative samples to create a triplet."""

    def __init__(
        self,
        margin: float,
        negative_selection: str = "semihard",
        perturb_labels: bool = True,
        perturb_labels_fraction: float = 0.5,
    ):
        """Constructor.

        Parameters
        ----------
        margin: float
            Triplet loss margin.
        negative_selection: str, default: "semihard"
            Method for negative selection: {"semihard", "hardest", "random"}.
        perturb_labels: bool, default: True
            Whether to perturb the ontology labels by coarse graining one level up.
        perturb_labels_fraction: float, default: 0.5
            The fraction of labels to perturb.

        Examples
        --------
        >>> triplet_selector = TripletSelector(margin=0.05,
                negative_selection="semihard",
                perturb_labels=True,
                perturb_labels_fraction=0.5,
            )
        """

        self.margin = margin
        self.negative_selection = negative_selection

        self.onto = import_cell_ontology()
        self.id2name = get_id_mapper(self.onto)
        self.name2id = {value: key for key, value in self.id2name.items()}

        self.perturb_labels = perturb_labels
        self.perturb_labels_fraction = perturb_labels_fraction

    def get_triplets_idx(
        self,
        embeddings: Union[np.ndarray, torch.Tensor],
        labels: Union[np.ndarray, torch.Tensor],
        int2label: dict,
        studies: Optional[Union[np.ndarray, torch.Tensor, list]] = None,
    ):
        """Get triplets as anchor, positive, and negative cell indices.

        Parameters
        ----------
        embeddings: numpy.ndarray, torch.Tensor
            Cell embeddings.
        labels: numpy.ndarray, torch.Tensor
            Cell labels in integer form.
        int2label: dict
            Dictionary to map labels in integer form to string
        studies: numpy.ndarray, torch.Tensor, optional, default: None
            Studies metadata for each cell.

        Returns
        -------
        triplets: Tuple[List, List, List]
            A tuple of lists containing anchor, positive, and negative cell indices.
        num_hard_triplets: int
            Number of hard triplets.
        num_viable_triplets: int
            Number of viable triplets.
        )
        """

        if isinstance(embeddings, torch.Tensor):
            distance_matrix = self.pdist(embeddings.detach().cpu().numpy())
        else:
            distance_matrix = self.pdist(embeddings)

        if isinstance(labels, torch.Tensor):
            labels = labels.detach().cpu().numpy()

        if studies is not None and isinstance(studies, torch.Tensor):
            studies = studies.detach().cpu().numpy()

        labels_ids = np.array([self.name2id[int2label[label]] for label in labels])
        labels_set = set(labels)

        if self.perturb_labels:
            labels_ids_set = set(labels_ids.tolist())
            label2int = {value: key for key, value in int2label.items()}
            perturb_list = random.choices(
                np.arange(len(labels)),
                k=int(len(labels) * self.perturb_labels_fraction),
            )
            for i in perturb_list:  # cells chosen for perturbation of labels
                term_id = self.name2id[int2label[labels[i]]]
                parents = set()
                max_ancestors = (
                    1  # range of ancestor levels. 1: parents, 2: grandparents, ...
                )
                ancestor_level = 0
                while ancestor_level < max_ancestors:
                    ancestor_level += 1
                    if not parents:
                        parents = get_parents(self.onto, term_id)
                    else:
                        current = set()
                        for p in parents:
                            current = current | get_parents(self.onto, p)
                        parents = current
                    found = any((parent in labels_ids_set for parent in parents))
                    if found is True:
                        parents = list(parents)
                        np.random.shuffle(parents)
                        p = next(
                            parent for parent in parents if parent in labels_ids_set
                        )
                        labels[i] = label2int[self.id2name[p]]
                        break  # label perturbed, skip the rest of the ancestors

        triplets = []
        num_hard_triplets = 0
        num_viable_triplets = 0
        for label in labels_set:
            term_id = self.name2id[int2label[label]]
            ancestors = get_all_ancestors(self.onto, term_id)
            descendants = get_all_descendants(self.onto, term_id)
            violating_terms = ancestors.union(descendants)

            label_mask = labels == label
            label_indices = np.where(label_mask)[0]
            if len(label_indices) < 2:
                continue

            # negatives are labels that are not the same as current and not in violating terms
            negative_indices = np.where(
                np.logical_not(label_mask | np.isin(labels_ids, list(violating_terms)))
            )[0]

            # compute all pairs of anchor-positives
            anchor_positives = list(combinations(label_indices, 2))

            # enforce anchor and positive coming from different studies
            if studies is not None:
                anchor_positives = [
                    (anchor, positive)
                    for anchor, positive in anchor_positives
                    if studies[anchor] != studies[positive]
                ]

            for anchor_positive in anchor_positives:
                loss_values = (
                    distance_matrix[anchor_positive[0], anchor_positive[1]]
                    - distance_matrix[[anchor_positive[0]], negative_indices]
                    + self.margin
                )
                num_hard_triplets += (loss_values > 0).sum()
                num_viable_triplets += loss_values.size

                # select one negative for anchor positive pair based on selection function
                if self.negative_selection == "semihard":
                    hard_negative = self.semihard_negative(loss_values)
                elif self.negative_selection == "hardest":
                    hard_negative = self.hardest_negative(loss_values)
                elif self.negative_selection == "random":
                    hard_negative = self.random_negative(loss_values)
                else:
                    hard_negative = None

                if hard_negative is not None:
                    hard_negative = negative_indices[hard_negative]
                    triplets.append(
                        [anchor_positive[0], anchor_positive[1], hard_negative]
                    )

        if len(triplets) == 0:
            triplets.append([0, 0, 0])

        anchor_idx, positive_idx, negative_idx = tuple(
            map(list, zip(*triplets))
        )  # tuple([list(t) for t in zip(*triplets)])

        return (
            (
                anchor_idx,
                positive_idx,
                negative_idx,
            ),
            num_hard_triplets,
            num_viable_triplets,
        )


    def get_triplets(
        self,
        embeddings: Union[np.ndarray, torch.Tensor],
        labels: Union[np.ndarray, torch.Tensor],
        int2label: dict,
        studies: Optional[Union[np.ndarray, torch.Tensor, list]] = None,
    ):
        """Get triplets as anchor, positive, and negative cell embeddings.

        Parameters
        ----------
        embeddings: numpy.ndarray, torch.Tensor
            Cell embeddings.
        labels: numpy.ndarray, torch.Tensor
            Cell labels in integer form.
        int2label: dict
            Dictionary to map labels in integer form to string
        studies: numpy.ndarray, torch.Tensor, optional, default: None
            Studies metadata for each cell.

        Returns
        -------
        triplets: Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]
            A tuple of numpy arrays containing anchor, positive, and negative cell embeddings.
        num_hard_triplets: int
            Number of hard triplets.
        num_viable_triplets: int
            Number of viable triplets.
        """

        (
            triplets_idx,
            num_hard_triplets,
            num_viable_triplets,
        ) = self.get_triplets_idx(embeddings, labels, int2label, studies)
        anchor_idx, positive_idx, negative_idx = triplets_idx
        return (
            (
                embeddings[anchor_idx],
                embeddings[positive_idx],
                embeddings[negative_idx],
            ),
            num_hard_triplets,
            num_viable_triplets,
        )


    def pdist(self, vectors: np.ndarray):
        """Get pair-wise distance between all cell embeddings.

        Parameters
        ----------
        vectors: numpy.ndarray
            Cell embeddings.

        Returns
        -------
        numpy.ndarray
            Distance matrix of cell embeddings.
        """

        vectors_squared_sum = (vectors**2).sum(axis=1)
        distance_matrix = (
            -2 * np.matmul(vectors, np.matrix.transpose(vectors))
            + vectors_squared_sum.reshape(1, -1)
            + vectors_squared_sum.reshape(-1, 1)
        )
        return distance_matrix


    def hardest_negative(self, loss_values):
        """Get hardest negative.

        Parameters
        ----------
        loss_values: numpy.ndarray
            Triplet loss of all negatives for given anchor positive pair.

        Returns
        -------
        int
            Index of selection.
        """

        hard_negative = np.argmax(loss_values)
        return hard_negative if loss_values[hard_negative] > 0 else None


    def random_negative(self, loss_values):
        """Get random negative.

        Parameters
        ----------
        loss_values: numpy.ndarray
            Triplet loss of all negatives for given anchor positive pair.

        Returns
        -------
        int
            Index of selection.
        """

        hard_negatives = np.where(loss_values > 0)[0]
        return np.random.choice(hard_negatives) if len(hard_negatives) > 0 else None


    def semihard_negative(self, loss_values):
        """Get a random semihard negative.

        Parameters
        ----------
        loss_values: numpy.ndarray
            Triplet loss of all negatives for given anchor positive pair.

        Returns
        -------
        int
            Index of selection.
        """

        semihard_negatives = np.where(
            np.logical_and(loss_values < self.margin, loss_values > 0)
        )[0]
        return (
            np.random.choice(semihard_negatives)
            if len(semihard_negatives) > 0
            else None
        )


    def get_asw(
        self,
        embeddings: Union[np.ndarray, torch.Tensor],
        labels: List[str],
        int2label: dict,
        metric: str = "cosine",
    ) -> float:
        """Get the average silhouette width of celltypes, being aware of cell ontology such that
           ancestors are not considered inter-cluster and descendants are considered intra-cluster.

        Parameters
        ----------
        embeddings: numpy.ndarray, torch.Tensor
            Cell embeddings.
        labels: List[str]
            Celltype names.
        int2label: dict
            Dictionary to map labels in integer form to string
        metric: str, default: "cosine"
            The distance metric to use for scipy.spatial.distance.cdist().

        Returns
        -------
        asw: float
            The average silhouette width.

        Examples
        --------
        >>> asw = ontology_silhouette_width(embeddings, labels, metric="cosine")
        """

        if isinstance(embeddings, torch.Tensor):
            distance_matrix = self.pdist(embeddings.detach().cpu().numpy())
        else:
            distance_matrix = self.pdist(embeddings)

        if isinstance(labels, torch.Tensor):
            labels = labels.detach().cpu().numpy()

        sw = []
        for i, label1 in enumerate(labels):
            term_id1 = self.name2id[int2label[label1]]
            ancestors = get_all_ancestors(self.onto, term_id1)
            descendants = get_all_descendants(self.onto, term_id1)

            a_i = []
            b_i = {}
            for j, label2 in enumerate(labels):
                if i == j:
                    continue

                term_id2 = self.name2id[int2label[label2]]
                if term_id2 == term_id1 or term_id2 in descendants:  # intra-cluster
                    a_i.append(distance_matrix[i, j])
                elif (
                    term_id2 != term_id1 and term_id2 not in ancestors
                ):  # inter-cluster
                    if term_id2 not in b_i:
                        b_i[term_id2] = []
                    b_i[term_id2].append(distance_matrix[i, j])

            if len(a_i) <= 1 or not b_i:
                continue
            a_i = np.sum(a_i) / (len(a_i) - 1)
            b_i = np.min(
                [
                    np.sum(values) / len(values)
                    for values in b_i.values()
                    if len(values) > 1
                ]
            )

            s_i = (b_i - a_i) / np.max([a_i, b_i])
            sw.append(s_i)
        return np.mean(sw)

In [48]:
triplet_selector = TripletSelector(margin=0.05,
                 negative_selection="semihard",
                 perturb_labels=True,
                 perturb_labels_fraction=0.5 )

In [49]:
int2label

{0: 'oligodendrocyte',
 1: 'lymphocyte',
 2: 'oligodendrocyte precursor cell',
 3: 'phagocyte',
 4: 'endothelial cell',
 5: 'microglial cell',
 6: 'neuron',
 7: 'astrocyte'}

In [50]:
label2int

{'oligodendrocyte': 0,
 'lymphocyte': 1,
 'oligodendrocyte precursor cell': 2,
 'phagocyte': 3,
 'endothelial cell': 4,
 'microglial cell': 5,
 'neuron': 6,
 'astrocyte': 7}

In [51]:
int2label  # 这是一个形状为 (N,) 的数组或张量，包含每个样本的标签

{0: 'oligodendrocyte',
 1: 'lymphocyte',
 2: 'oligodendrocyte precursor cell',
 3: 'phagocyte',
 4: 'endothelial cell',
 5: 'microglial cell',
 6: 'neuron',
 7: 'astrocyte'}

In [52]:
len(triplet_selector.id2name)

2810

In [53]:
triplet_selector.name2id.get('astrocyte')

'CL:0000127'

In [54]:
triplet_selector.name2id

{'cell': 'CL:0000000',
 'primary cultured cell': 'CL:0000001',
 'neural crest derived fibroblast': 'CL:0000005',
 'neuronal receptor cell': 'CL:0000006',
 'early embryonic cell (metazoa)': 'CL:0000007',
 'migratory cranial neural crest cell': 'CL:0000008',
 'cultured cell': 'CL:0000010',
 'migratory trunk neural crest cell': 'CL:0000011',
 'germ line stem cell': 'CL:0000014',
 'male germ cell': 'CL:0000015',
 'male germ line stem cell': 'CL:0000016',
 'spermatocyte': 'CL:0000017',
 'spermatid': 'CL:0000018',
 'sperm': 'CL:0000019',
 'spermatogonium': 'CL:0000020',
 'female germ cell': 'CL:0000021',
 'female germ line stem cell': 'CL:0000022',
 'oocyte': 'CL:0000023',
 'oogonial cell': 'CL:0000024',
 'egg cell': 'CL:0000025',
 'invertebrate nurse cell': 'CL:0000026',
 'smooth muscle cell neural crest derived': 'CL:0000027',
 'CNS neuron (sensu Nematoda and Protostomia)': 'CL:0000028',
 'neural crest derived neuron': 'CL:0000029',
 'glioblast': 'CL:0000030',
 'neuroblast (sensu Vertebrat

In [55]:
from itertools import combinations
import numpy as np
import random
import torch
from typing import List, Union, Optional

from scimilarity.ontologies import (
    import_cell_ontology,
    get_id_mapper,
    get_all_ancestors,
    get_all_descendants,
    get_parents,
)

In [56]:
from datetime import datetime
import json
import os
import pandas as pd
import pytorch_lightning as pl
import torch
from torch import nn
import torch.nn.functional as F
from typing import Optional, List

from scimilarity.triplet_selector import TripletSelector
from scimilarity.nn_models import Encoder, Decoder

class TripletLoss(torch.nn.TripletMarginLoss):
    """
    Wrapper for pytorch TripletMarginLoss.
    Triplets are generated using TripletSelector object which take embeddings and labels
    then return triplets.
    """

    def __init__(
        self,
        margin: float,
        sample_across_studies: bool = True,
        negative_selection: str = "semihard",
        perturb_labels: bool = True,
        perturb_labels_fraction: float = 0.5,
    ):
        """Constructor.

        Parameters
        ----------
        margin: float
            Triplet loss margin.
        sample_across_studies: bool, default: True
            Whether to enforce anchor-positive pairs being from different studies.
        negative_selection: str
            Method for negative selection: {"semihard", "hardest", "random"}
        perturb_labels: bool, default: True
            Whether to perturb the ontology labels by coarse graining one level up.
        perturb_labels_fraction: float, default: 0.5
            The fraction of labels to perturb

        Examples
        --------
        >>> triplet_loss = TripletLoss(margin=0.05)
        """

        super().__init__()
        self.margin = margin
        self.sample_across_studies = sample_across_studies
        self.triplet_selector = TripletSelector(
            margin=margin,
            negative_selection=negative_selection,
            perturb_labels=perturb_labels,
            perturb_labels_fraction=perturb_labels_fraction,
        )

    def forward(
        self,
        embeddings: torch.Tensor,
        labels: torch.Tensor,
        int2label: dict,
        studies: torch.Tensor,
    ):
        if self.sample_across_studies is False:
            studies = None

        (
            triplets,
            num_violating_triplets,
            num_viable_triplets,
        ) = self.triplet_selector.get_triplets(embeddings, labels, int2label, studies)

        anchor, positive, negative = triplets
        return (
            F.triplet_margin_loss(
                anchor,
                positive,
                negative,
                margin=self.margin,
                p=self.p,
                eps=self.eps,
                swap=self.swap,
                reduction="none",
            ),
            torch.tensor(num_violating_triplets, dtype=torch.float),
            torch.tensor(num_viable_triplets, dtype=torch.float),
        )


In [57]:
triplet_loss = TripletLoss(margin=0.05)

In [58]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [59]:
class MetricLearning(pl.LightningModule):
    """A class encapsulating the metric learning."""

    def __init__(
        self,
        n_genes: int,
        latent_dim: int = 128,
        hidden_dim: List[int] = [1024, 1024],
        dropout: float = 0.5,
        input_dropout: float = 0.4,
        triplet_loss_weight: float = 0.001,
        margin: float = 0.05,
        negative_selection: str = "semihard",
        sample_across_studies: bool = True,
        perturb_labels: bool = True,
        perturb_labels_fraction: float = 0.5,
        lr: float = 5e-3,
        l1: float = 1e-4,
        l2: float = 0.01,
        max_epochs: int = 500,
        residual: bool = False,
    ):
        """Constructor.

        Parameters (network)
        ----------
        n_genes: int
            The number of genes in the gene space, representing the input dimensions.
        latent_dim: int, default: 128
            The latent space dimensions
        hidden_dim: List[int], default: [1024, 1024]
            A list of hidden layer dimensions, describing the number of layers and their dimensions.
            Hidden layers are constructed in the order of the list for the encoder and in reverse
            for the decoder.
        dropout: float, default: 0.5
            The dropout rate for hidden layers
        input_dropout: float, default: 0.4
            The dropout rate for the input layer

        Parameters (training)
        ----------
        triplet_loss_weight: float, default 0.001
            The weighting for triplet loss vs reconstruction loss.  This weighting sums to 1
            such that triplet loss weight is triplet_loss_weight and reconstruction loss weight is (1 - triplet_loss_weight).
        margin: float, default: 0.05
            The margin parameter in triplet loss.
        negative_selection: {"semihard", "hardest", "random"}, default: "semihard"
            The negative selection function.
        sample_across_studies: bool, default: True
            Whether to enforce anchor-positive pairs being from different studies.
        perturb_labels: bool, default: True
            Whether to perturb celltype labels by coarse graining the label based on cell ontology.
        perturb_labels_fraction: float, default: 0.5
            The fraction of cells per batch to perform label perturbation.
        lr: float, default: 5e-3
            The initial learning rate
        l1: float, default: 1e-4
            The l1 penalty lambda. A value of 0 will disable l1 penalty.
        l2: float, default: 1e-2
            The l2 penalty lambda (weight decay). A value of 0 will disable l2 penalty.
        max_epochs: int, default: 500
            The max epochs, used by the scheduler to determine lr annealing rate.

        Examples
        --------
        >>> datamodule = MetricLearningZarrDataModule(
                batch_size=1000,
                num_workers=1,
                obs_field="celltype_name",
                train_path="train",
                gene_order="gene_order.tsv",
            )
        >>> model = MetricLearning(datamodule.n_genes)
        """

        super().__init__()
        self.save_hyperparameters()
        valid_negative_selection = {"semihard", "hardest", "random"}
        if negative_selection not in valid_negative_selection:
            raise ValueError(
                f"Unknown return_type {negative_selection}."
                f"Options are: {valid_negative_selection}."
            )

        # network architecture
        self.n_genes = n_genes
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.input_dropout = input_dropout
        self.residual = residual

        # networks
        self.encoder = Encoder(
            self.n_genes,
            latent_dim=self.latent_dim,
            hidden_dim=self.hidden_dim,
            dropout=self.dropout,
            input_dropout=self.input_dropout,
            residual=self.residual,
        )
        self.decoder = Decoder(
            self.n_genes,
            latent_dim=self.latent_dim,
            hidden_dim=list(reversed(self.hidden_dim)),
            dropout=self.dropout,
            residual=self.residual,
        )

        # save layer sizes
        model_state_dict = self.encoder.state_dict()
        self.layer_sizes = {
            entry: list(model_state_dict[entry].size()) for entry in model_state_dict
        }

        # mixed loss weight
        self.triplet_loss_weight = triplet_loss_weight

        # constraints
        self.margin = margin
        self.negative_selection = negative_selection
        self.sample_across_studies = sample_across_studies
        self.perturb_labels = perturb_labels
        self.perturb_labels_fraction = perturb_labels_fraction

        # lr and regularization
        self.lr = lr
        self.l1 = l1
        self.l2 = l2
        self.max_epochs = max_epochs

        # losses
        self.triplet_loss_fn = TripletLoss(
            margin=self.margin,
            negative_selection=self.negative_selection,
            sample_across_studies=self.sample_across_studies,
            perturb_labels=self.perturb_labels,
            perturb_labels_fraction=self.perturb_labels_fraction,
        )
        self.mse_loss_fn = nn.MSELoss()

        self.scheduler = None

        self.val_step_outputs = []
        self.test_step_outputs = []

    def configure_optimizers(self):
        """Configure optimizers."""

        optimizer = torch.optim.AdamW(self.parameters(), self.lr, weight_decay=self.l2)
        self.scheduler = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=self.max_epochs
            ),
            "interval": "epoch",
            "frequency": 1,
        }
        return {
            "optimizer": optimizer,
            "lr_scheduler": self.scheduler,
        }  # pytorch-lightning required format


    def forward(self, x):
        """Forward.

        Parameters
        ----------
        x: torch.Tensor
            Input tensor corresponding to input layer.

        Returns
        -------
        z: torch.Tensor
            Output tensor corresponding to the last encoder layer.
        x_hat: torch.Tensor
            Output tensor corresponding to the last decoder layer.
        """

        z = self.encoder(x)
        x_hat = self.decoder(z)
        return z, x_hat


    def get_losses(self, batch, use_studies: bool = True):
        """Calculate the triplet and reconstruction loss.

        Parameters
        ----------
        batch:
            A batch as defined by a pytorch DataLoader.
        use_studies: bool, default: True
            Whether to use studies metadata in mining triplets and calculating triplet loss

        Returns
        -------
        triplet_loss: torch.Tensor
            Triplet loss.
        mse: torch.Tensor
            MSE reconstruction loss
        num_hard_triplets: torch.Tensor
            Number of hard triplets.
        num_viable_triplets: torch.Tensor
            Number of viable triplets.
        """

        cells, labels, studies = batch
        if not use_studies:
            studies = None
        embedding, reconstruction = self(cells)
        triplet_loss, num_hard_triplets, num_viable_triplets = self.triplet_loss_fn(
            embedding, labels, int2label, studies
        )
        mse = self.mse_loss_fn(cells, reconstruction)
        asw = self.triplet_loss_fn.triplet_selector.get_asw(
            embedding, labels, int2label
        )
        nmse = mse / cells.pow(2).mean()
        return (
            triplet_loss,
            mse,
            num_hard_triplets,
            num_viable_triplets,
            asw,
            nmse,
        )

    def get_mixed_loss(self, triplet_loss, mse):
        """Calculate the mixed loss.

        Parameters
        ----------
        triplet_loss: torch.Tensor
            Triplet loss.
        mse: torch.Tensor
            MSE reconstruction loss

        Returns
        -------
        torch.Tensor
            Mixed loss.
        """

        if self.triplet_loss_weight == 0:
            return mse
        if self.triplet_loss_weight == 1:
            return triplet_loss
        return (self.triplet_loss_weight * triplet_loss) + (
            (1.0 - self.triplet_loss_weight) * mse
        )


    def training_step(self, batch, batch_idx):
        """Pytorch-lightning training step.

        Parameters
        ----------
        batch:
            A batch as defined by a pytorch DataLoader.
        batch_idx:
            A batch index as defined by a pytorch-lightning.
        """

        (
            triplet_losses,
            mse,
            num_hard_triplets,
            num_viable_triplets,
            asw,
            nmse,
        ) = self.get_losses(batch)

        triplet_loss = triplet_losses.mean()
        num_nonzero_loss = (triplet_losses > 0).sum(dtype=torch.float).detach()
        hard_triplets = num_hard_triplets / num_viable_triplets

        loss = self.get_mixed_loss(triplet_loss, mse)

        current_lr = self.scheduler["scheduler"].get_last_lr()[0]

        if self.l1 > 0:  # use l1 penalty for first layer
            for layer in self.encoder.network:
                if isinstance(layer, nn.Linear):
                    l1_norm = sum(p.abs().sum() for p in layer.parameters())
                    l1_penalty = self.l1 * l1_norm * current_lr
                    loss += l1_penalty
                    self.log(
                        "train l1 penalty", l1_penalty, prog_bar=False, logger=True
                    )
                    break

        # if self.l2 > 0:  # use l2 penalty
        #    l2_regularization = []
        #    for layer in self.encoder.network:
        #        if isinstance(layer, nn.Linear):
        #            l2_norm = sum(p.pow(2).sum() for p in layer.parameters())
        #            l2_regularization.append(l2_norm)
        #    for layer in self.decoder.network:
        #        if isinstance(layer, nn.Linear):
        #            l2_norm = sum(p.pow(2).sum() for p in layer.parameters())
        #            l2_regularization.append(l2_norm)
        #    l2_penalty = (
        #        self.l2 * sum(l2_regularization[0:-1]) * current_lr
        #    )  # all but reconstruction layer
        #    loss += l2_penalty
        #    self.log("train l2 penalty", l2_penalty, prog_bar=False, logger=True)

        self.log("train loss", loss, prog_bar=False, logger=True)
        self.log("train triplet loss", triplet_loss, prog_bar=True, logger=True)
        self.log("train mse", mse, prog_bar=True, logger=True)
        self.log("train hard triplets", hard_triplets, prog_bar=True, logger=True)
        self.log(
            "train num nonzero loss", num_nonzero_loss, prog_bar=False, logger=True
        )
        self.log(
            "train num hard triplets", num_hard_triplets, prog_bar=False, logger=True
        )
        self.log(
            "train num viable triplets",
            num_viable_triplets,
            prog_bar=False,
            logger=True,
        )
        self.log("train asw", asw, prog_bar=True, logger=True)
        self.log("train nmse", nmse, prog_bar=True, logger=True)
        return {
            "loss": loss,
            "triplet_loss": triplet_loss.detach(),
            "mse": mse.detach(),
            "hard_triplets": hard_triplets,
            "num_nonzeros_loss": num_nonzero_loss,
            "num_hard_triplets": num_hard_triplets,
            "num_viable_triplets": num_viable_triplets,
            "asw": asw,
            "nmse": nmse,
        }


    def on_validation_epoch_start(self):
        """Pytorch-lightning validation epoch start."""
        super().on_validation_epoch_start()
        self.val_step_outputs = []


    def validation_step(self, batch, batch_idx):
        """Pytorch-lightning validation step.

        Parameters
        ----------
        batch:
            A batch as defined by a pytorch DataLoader.
        batch_idx:
            A batch index as defined by a pytorch-lightning.
        """

        if self.trainer.datamodule.val_dataset is None:
            return {}
        return self._eval_step(batch, prefix="val")


    def on_validation_epoch_end(self):
        """Pytorch-lightning validation epoch end evaluation."""

        if self.trainer.datamodule.val_dataset is None:
            return {}
        return self._eval_epoch(prefix="val")


    def on_test_epoch_start(self):
        """Pytorch-lightning test epoch start."""
        super().on_test_epoch_start()
        self.test_step_outputs = []


    def test_step(self, batch, batch_idx):
        """Pytorch-lightning test step.

        Parameters
        ----------
        batch:
            A batch as defined by a pytorch DataLoader.
        batch_idx:
            A batch index as defined by a pytorch-lightning.
        """

        if self.trainer.datamodule.val_dataset is None:
            return {}
        return self._eval_step(batch, prefix="test")


    def on_test_epoch_end(self):
        """Pytorch-lightning test epoch end evaluation."""

        if self.trainer.datamodule.val_dataset is None:
            return {}
        return self._eval_epoch(prefix="test")


    def _eval_step(self, batch, prefix: str):
        """Evaluation of validation or test step.

        Parameters
        ----------
        batch:
            A batch as defined by a pytorch DataLoader.
        prefix: str
            A string prefix to label validation versus test evaluation.

        Returns
        -------
        dict
            A dictionary containing step evaluation metrics.
        """

        (
            triplet_losses,
            mse,
            num_hard_triplets,
            num_viable_triplets,
            asw,
            nmse,
        ) = self.get_losses(batch, use_studies=False)

        triplet_loss = triplet_losses.mean()
        num_nonzero_loss = (triplet_losses > 0).sum()
        hard_triplets = num_hard_triplets / num_viable_triplets

        loss = self.get_mixed_loss(triplet_loss, mse)

        evaluation_metric = (1 - asw) / 2 + nmse

        losses = {
            f"{prefix}_loss": loss,
            f"{prefix}_triplet_loss": triplet_loss,
            f"{prefix}_mse": mse,
            f"{prefix}_hard_triplets": hard_triplets,
            f"{prefix}_num_nonzero_loss": num_nonzero_loss,
            f"{prefix}_num_hard_triplets": num_hard_triplets,
            f"{prefix}_num_viable_triplets": num_viable_triplets,
            f"{prefix}_num_viable_triplets": num_viable_triplets,
            f"{prefix}_asw": asw,
            f"{prefix}_nmse": nmse,
            f"{prefix}_evaluation_metric": evaluation_metric,
        }

        if prefix == "val":
            self.val_step_outputs.append(losses)
        elif prefix == "test":
            self.test_step_outputs.append(losses)
        return losses

    def _eval_epoch(self, prefix: str):
        """Evaluation of validation or test epoch.

        Parameters
        ----------
        prefix: str
            A string prefix to label validation versus test evaluation.

        Returns
        -------
        dict
            A dictionary containing epoch evaluation metrics.
        """

        if prefix == "val":
            step_outputs = self.val_step_outputs
        elif prefix == "test":
            step_outputs = self.test_step_outputs

        loss = torch.Tensor([step[f"{prefix}_loss"] for step in step_outputs]).mean()
        triplet_loss = torch.Tensor(
            [step[f"{prefix}_triplet_loss"] for step in step_outputs]
        ).mean()
        mse = torch.Tensor([step[f"{prefix}_mse"] for step in step_outputs]).mean()
        hard_triplets = torch.Tensor(
            [step[f"{prefix}_hard_triplets"] for step in step_outputs]
        ).mean()
        num_nonzero_loss = torch.Tensor(
            [step[f"{prefix}_num_nonzero_loss"] for step in step_outputs]
        ).mean()
        num_hard_triplets = torch.Tensor(
            [step[f"{prefix}_num_hard_triplets"] for step in step_outputs]
        ).mean()
        num_viable_triplets = torch.Tensor(
            [step[f"{prefix}_num_viable_triplets"] for step in step_outputs]
        ).mean()
        asw = torch.Tensor([step[f"{prefix}_asw"] for step in step_outputs]).mean()
        nmse = torch.Tensor([step[f"{prefix}_nmse"] for step in step_outputs]).mean()
        evaluation_metric = torch.Tensor(
            [step[f"{prefix}_evaluation_metric"] for step in step_outputs]
        ).mean()

        self.log(f"{prefix} loss", loss, logger=True)
        self.log(f"{prefix} triplet loss", triplet_loss, logger=True)
        self.log(f"{prefix} mse", mse, logger=True)
        self.log(f"{prefix} hard triplets", hard_triplets, logger=True)
        self.log(f"{prefix} num nonzero loss", num_nonzero_loss, logger=True)
        self.log(f"{prefix} num hard triplets", num_hard_triplets, logger=True)
        self.log(f"{prefix} num viable triplets", num_viable_triplets, logger=True)
        self.log(f"{prefix} asw", asw, logger=True)
        self.log(f"{prefix} nmse", nmse, logger=True)
        self.log(f"{prefix} evaluation_metric", evaluation_metric, logger=True)

        losses = {
            f"{prefix}_loss": loss,
            f"{prefix}_triplet_loss": triplet_loss,
            f"{prefix}_mse": mse,
            f"{prefix}_hard_triplets": hard_triplets,
            f"{prefix}_num_nonzero_loss": num_nonzero_loss,
            f"{prefix}_num_hard_triplets": num_hard_triplets,
            f"{prefix}_num_viable_triplets": num_viable_triplets,
            f"{prefix}_asw": asw,
            f"{prefix}_nmse": nmse,
            f"{prefix}_evaluation_metric": evaluation_metric,
        }
        return losses

    def save_all(
        self,
        model_path: str,
    ):
        if not os.path.isdir(model_path):
            os.makedirs(model_path)

        # save NN model
        self.encoder.save_state(os.path.join(model_path, "encoder.ckpt"))
        self.decoder.save_state(os.path.join(model_path, "decoder.ckpt"))

        # save layer sizes as json, useful to infer model architecture
        with open(os.path.join(model_path, "layer_sizes.json"), "w") as f:
            f.write(json.dumps(self.layer_sizes))

        # save hyperparameters as json
        hyperparameters = {
            "latent_dim": self.latent_dim,
            "hidden_dim": self.hidden_dim,
            "dropout": self.dropout,
            "input_dropout": self.input_dropout,
            "margin": self.margin,
            "triplet_loss_weight": self.triplet_loss_weight,
            "negative_selection": self.negative_selection,
            "sample_across_studies": self.sample_across_studies,
            "perturb_labels": self.perturb_labels,
            "perturb_labels_fraction": self.perturb_labels_fraction,
            "lr": self.lr,
            "l1_lambda": self.l1,
            "l2_lambda": self.l2,
            "batch_size": self.trainer.datamodule.batch_size,
            "max_epochs": self.max_epochs,
            "residual": self.residual,
        }
        with open(os.path.join(model_path, "hyperparameters.json"), "w") as f:
            f.write(json.dumps(hyperparameters))

        # write gene order
        with open(os.path.join(model_path, "gene_order.tsv"), "w") as f:
            f.write("\n".join(self.trainer.datamodule.gene_order))

        # write dictionary to map label_ints to labels
        pd.Series(self.trainer.datamodule.int2label).to_csv(
            os.path.join(model_path, "label_ints.csv")
        )

        # write metadata: data paths, timestamp
        meta_data = {
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        }
        if "train_path" in dir(self.trainer.datamodule):
            meta_data["train_path"] = self.trainer.datamodule.train_path
            meta_data["val_path"] = self.trainer.datamodule.val_path
            # write reference labels
            with open(os.path.join(model_path, "reference_labels.tsv"), "w") as f:
                f.write("\n".join(self.trainer.datamodule.train_Y))
        elif "cell_tdb_uri" in dir(self.trainer.datamodule):
            meta_data["cell_tdb_uri"] = self.trainer.datamodule.cell_tdb_uri
            meta_data["gene_tdb_uri"] = self.trainer.datamodule.gene_tdb_uri
            meta_data["counts_tdb_uri"] = self.trainer.datamodule.counts_tdb_uri
            self.trainer.datamodule.train_df.to_csv(
                os.path.join(model_path, "train_cells.csv")
            )
            if self.trainer.datamodule.val_df is not None:
                self.trainer.datamodule.val_df.to_csv(
                    os.path.join(model_path, "val_cells.csv")
                )
            # write reference labels
            with open(os.path.join(model_path, "reference_labels.tsv"), "w") as f:
                f.write(
                    "\n".join(
                        self.trainer.datamodule.train_df["cellTypeName"].values.tolist()
                    )
                )
        with open(os.path.join(model_path, "metadata.json"), "w") as f:
            f.write(json.dumps(meta_data))

    def load_state(
        self,
        encoder_filename: str,
        decoder_filename: str,
        use_gpu: bool = False,
        freeze: bool = False,
    ):
        """Load model state.

        Parameters
        ----------
        encoder_filename: str
            Filename containing the encoder model state.
        decoder_filename: str
            Filename containing the decoder model state.
        use_gpu: bool, default: False
            Boolean indicating whether or not to use GPUs.
        freeze: bool, default: False
            Freeze all but bottleneck layer, used if pretraining the encoder.
        """

        self.encoder.load_state(encoder_filename, use_gpu)
        self.decoder.load_state(decoder_filename, use_gpu)

        if freeze:
            # encoder batchnorm freeze
            for i in range(len(self.encoder.network)):
                if isinstance(self.encoder.network[i], nn.BatchNorm1d):
                    for param in self.encoder.network[i].parameters():
                        param.requires_grad = False  # freeze

            # encoder linear freeze
            encoder_linear_idx = []
            for i in range(len(self.encoder.network)):
                if isinstance(self.encoder.network[i], nn.Linear):
                    encoder_linear_idx.append(i)
            for i in range(len(encoder_linear_idx)):
                if i < len(encoder_linear_idx) - 1:  # freeze all but bottleneck
                    for param in self.encoder.network[
                        encoder_linear_idx[i]
                    ].parameters():
                        param.requires_grad = False  # freeze
                else:
                    for param in self.encoder.network[
                        encoder_linear_idx[i]
                    ].parameters():
                        param.requires_grad = True  # unfreeze

In [60]:
datamodule.n_genes

34667

In [61]:
from pytorch_lightning import Trainer

# 初始化你的模型
model = MetricLearning(datamodule.n_genes)

In [62]:
model

MetricLearning(
  (encoder): Encoder(
    (network): ModuleList(
      (0): Sequential(
        (0): Dropout(p=0.4, inplace=False)
        (1): Linear(in_features=34667, out_features=1024, bias=True)
        (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): PReLU(num_parameters=1)
      )
      (1): Sequential(
        (0): Dropout(p=0.5, inplace=False)
        (1): Linear(in_features=1024, out_features=1024, bias=True)
        (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): PReLU(num_parameters=1)
      )
      (2): Linear(in_features=1024, out_features=128, bias=True)
    )
  )
  (decoder): Decoder(
    (network): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=128, out_features=1024, bias=True)
        (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
      )
      (1): Sequential(
        (

In [63]:
model.forward(batch[0])

(tensor([[ 0.0706,  0.2073,  0.0109,  ..., -0.0017, -0.0100, -0.0760],
         [-0.0128, -0.0461, -0.1440,  ...,  0.0762,  0.0803, -0.1886],
         [ 0.1080,  0.0219, -0.0311,  ...,  0.1358, -0.0655,  0.0179],
         ...,
         [ 0.0171,  0.0853, -0.0644,  ...,  0.0506,  0.0892, -0.0411],
         [ 0.1707,  0.1254, -0.0641,  ...,  0.1234, -0.0903, -0.1283],
         [-0.0066, -0.0200,  0.0060,  ...,  0.0780, -0.0463, -0.0759]],
        grad_fn=<DivBackward0>),
 tensor([[-0.2675,  0.4704,  0.2071,  ...,  0.2816, -0.1630,  0.3185],
         [-0.4967,  0.4436,  0.2416,  ...,  0.1844,  0.3107, -0.0721],
         [-0.6210, -0.3960,  0.1267,  ...,  0.0835,  0.2555,  0.1837],
         ...,
         [-0.1788,  0.6497,  0.0054,  ..., -0.0246,  0.8695, -0.1197],
         [-0.0763, -0.6518, -0.6226,  ..., -0.2271,  0.1449, -0.1271],
         [ 0.3007,  0.3907, -0.1019,  ..., -0.1534,  0.0281, -0.2759]],
        grad_fn=<AddmmBackward0>))

In [64]:
model.forward(batch[0])[0].shape

torch.Size([1000, 128])

In [65]:
model.forward(batch[0])[1].shape

torch.Size([1000, 34667])

In [66]:
embeddings, _ = model.forward(batch[0])

In [67]:
embeddings

tensor([[ 0.0106,  0.1331,  0.0747,  ...,  0.2205,  0.0040, -0.1336],
        [ 0.0485,  0.0374, -0.0858,  ...,  0.1137,  0.0259, -0.1913],
        [ 0.0031,  0.0510,  0.0277,  ..., -0.0363, -0.0523, -0.2076],
        ...,
        [ 0.0592, -0.0116,  0.1152,  ..., -0.0365,  0.0745, -0.0992],
        [ 0.0536, -0.1053,  0.0034,  ...,  0.0659, -0.0403, -0.4034],
        [-0.1027, -0.0217,  0.0662,  ..., -0.0620, -0.1144, -0.1382]],
       grad_fn=<DivBackward0>)

In [68]:
triplet_selector.get_triplets(embeddings,labels,int2label)

((tensor([[ 1.0635e-02,  1.3310e-01,  7.4704e-02, -3.6707e-03,  7.4565e-04,
            1.0046e-01,  5.5280e-02, -1.7478e-01, -7.4276e-02,  9.0263e-02,
            3.8132e-02,  5.7911e-02,  1.8540e-01, -1.8250e-01, -8.2295e-02,
           -8.1861e-02,  2.2857e-02, -2.3162e-02,  5.9862e-02,  8.3447e-02,
            2.6331e-02, -3.8658e-02,  4.2412e-02, -3.5144e-02,  1.9559e-02,
           -1.3284e-01, -1.0458e-01,  1.1118e-02, -4.1271e-02,  1.4218e-01,
            6.7288e-02,  1.6084e-02,  9.6247e-02,  5.7376e-02,  5.7281e-02,
           -3.6550e-02,  8.1002e-02,  3.3958e-02,  3.3214e-02, -3.9106e-02,
           -1.8417e-01,  1.3401e-01, -4.0572e-02,  6.1037e-03,  1.3350e-01,
           -1.5692e-02, -2.4100e-01, -2.4445e-02, -5.1360e-02,  1.9058e-02,
           -5.9606e-02, -7.3374e-03,  1.2492e-01, -8.0148e-02, -3.2777e-02,
           -9.3750e-02,  1.1271e-01,  1.0915e-01,  3.5869e-02, -1.4058e-02,
           -2.4243e-02, -9.5141e-02, -1.3024e-01, -1.1425e-01, -8.4133e-02,
            

In [69]:
triplet_selector.get_triplets(embeddings,labels,int2label)[0]

(tensor([[ 1.0635e-02,  1.3310e-01,  7.4704e-02, -3.6707e-03,  7.4565e-04,
           1.0046e-01,  5.5280e-02, -1.7478e-01, -7.4276e-02,  9.0263e-02,
           3.8132e-02,  5.7911e-02,  1.8540e-01, -1.8250e-01, -8.2295e-02,
          -8.1861e-02,  2.2857e-02, -2.3162e-02,  5.9862e-02,  8.3447e-02,
           2.6331e-02, -3.8658e-02,  4.2412e-02, -3.5144e-02,  1.9559e-02,
          -1.3284e-01, -1.0458e-01,  1.1118e-02, -4.1271e-02,  1.4218e-01,
           6.7288e-02,  1.6084e-02,  9.6247e-02,  5.7376e-02,  5.7281e-02,
          -3.6550e-02,  8.1002e-02,  3.3958e-02,  3.3214e-02, -3.9106e-02,
          -1.8417e-01,  1.3401e-01, -4.0572e-02,  6.1037e-03,  1.3350e-01,
          -1.5692e-02, -2.4100e-01, -2.4445e-02, -5.1360e-02,  1.9058e-02,
          -5.9606e-02, -7.3374e-03,  1.2492e-01, -8.0148e-02, -3.2777e-02,
          -9.3750e-02,  1.1271e-01,  1.0915e-01,  3.5869e-02, -1.4058e-02,
          -2.4243e-02, -9.5141e-02, -1.3024e-01, -1.1425e-01, -8.4133e-02,
           5.9834e-03,  5

In [70]:
trainer = pl.Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [71]:
trainer

In [72]:
dir(trainer.train_dataloader)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [73]:
cells, labels, studies = batch

In [74]:
trainer.datamodule

In [75]:
datamodule.label2int

{'oligodendrocyte': 0,
 'lymphocyte': 1,
 'oligodendrocyte precursor cell': 2,
 'phagocyte': 3,
 'endothelial cell': 4,
 'microglial cell': 5,
 'neuron': 6,
 'astrocyte': 7}

In [76]:
# !pip install numpy==1.24.0

In [77]:
# import numpy

In [78]:
# numpy.__version__

In [79]:
# 开始训练
# trainer.fit(model, train_dataloader)

In [80]:
dir(trainer.datamodule)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [81]:
torch.save(model,"model.pth")

In [82]:
#####functions######

In [83]:
from typing import Optional, Tuple, Union

In [84]:
class CellEmbedding:
    """A class that embeds cell gene expression data using a ML model."""

    def __init__(
        self,
        model_path: str,
        use_gpu: bool = False,
    ):
        """Constructor.

        Parameters
        ----------
        model_path: str
            Path to the directory containing model files.
        use_gpu: bool, default: False
            Use GPU instead of CPU.

        Examples
        --------
        >>> ce = CellEmbedding(model_path="/opt/data/model")
        """

        import json
        import os
        import pandas as pd
        from .nn_models import Encoder

        self.model_path = model_path
        self.use_gpu = use_gpu

        self.filenames = {
            "model": os.path.join(self.model_path, "encoder.ckpt"),
            "gene_order": os.path.join(self.model_path, "gene_order.tsv"),
        }

        # get gene order
        with open(self.filenames["gene_order"], "r") as fh:
            self.gene_order = [line.strip() for line in fh]

        # get neural network model and infer network size
        with open(os.path.join(self.model_path, "layer_sizes.json"), "r") as fh:
            layer_sizes = json.load(fh)
        # keys: network.1.weight, network.2.weight, ..., network.n.weight
        layers = [
            (key, layer_sizes[key])
            for key in sorted(list(layer_sizes.keys()))
            if "weight" in key and len(layer_sizes[key]) > 1
        ]
        parameters = {
            "latent_dim": layers[-1][1][0],  # last
            "hidden_dim": [layer[1][0] for layer in layers][0:-1],  # all but last
        }

        self.n_genes = len(self.gene_order)
        self.latent_dim = parameters["latent_dim"]
        self.model = Encoder(
            n_genes=self.n_genes,
            latent_dim=parameters["latent_dim"],
            hidden_dim=parameters["hidden_dim"],
            residual=False,
        )
        if self.use_gpu is True:
            self.model.cuda()
        self.model.load_state(self.filenames["model"])
        self.model.eval()

        self.int2label = pd.read_csv(
            os.path.join(self.model_path, "label_ints.csv"), index_col=0
        )["0"].to_dict()
        self.label2int = {value: key for key, value in self.int2label.items()}

    def get_embeddings(
        self,
        X: Union["scipy.sparse.csr_matrix", "scipy.sparse.csc_matrix", "numpy.ndarray"],
        num_cells: int = -1,
        buffer_size: int = 10000,
    ) -> "numpy.ndarray":
        """Calculate embeddings for lognormed gene expression matrix.

        Parameters
        ----------
        X: scipy.sparse.csr_matrix, scipy.sparse.csc_matrix, numpy.ndarray
            Gene space aligned and log normalized (tp10k) gene expression matrix.
        num_cells: int, default: -1
            The number of cells to embed, starting from index 0.
            A value of -1 will embed all cells.
        buffer_size: int, default: 10000
            The number of cells to embed in one batch.

        Returns
        -------
        numpy.ndarray
            A 2D numpy array of embeddings [num_cells x latent_space_dimensions].

        Examples
        --------
        >>> from scimilarity.utils import align_dataset, lognorm_counts
        >>> ce = CellEmbedding(model_path="/opt/data/model")
        >>> data = align_dataset(data, ce.gene_order)
        >>> data = lognorm_counts(data)
        >>> embeddings = ce.get_embeddings(data.X)
        """

        import numpy as np
        from scipy.sparse import csr_matrix, csc_matrix
        import torch
        import zarr

        if num_cells == -1:
            num_cells = X.shape[0]

        if (
            (isinstance(X, csr_matrix) or isinstance(X, csc_matrix))
            and (
                isinstance(X.data, zarr.core.Array)
                or isinstance(X.indices, zarr.core.Array)
                or isinstance(X.indptr, zarr.core.Array)
            )
            and num_cells <= buffer_size
        ):
            X.data = X.data[...]
            X.indices = X.indices[...]
            X.indptr = X.indptr[...]

        embedding_parts = []
        with torch.inference_mode():  # disable gradients, not needed for inference
            for i in range(0, num_cells, buffer_size):
                profiles = None
                if isinstance(X, np.ndarray):
                    profiles = torch.Tensor(X[i : i + buffer_size])
                elif isinstance(X, torch.Tensor):
                    profiles = X[i : i + buffer_size]
                elif isinstance(X, csr_matrix) or isinstance(X, csc_matrix):
                    profiles = torch.Tensor(X[i : i + buffer_size].toarray())

                if profiles is None:
                    raise RuntimeError(f"Unknown data type {type(X)}.")

                if self.use_gpu is True:
                    profiles = profiles.cuda()
                embedding_parts.append(self.model(profiles))

        if not embedding_parts:
            raise RuntimeError(f"No valid cells detected.")

        if self.use_gpu:
            # detach, move from gpu into cpu, return as numpy array
            embedding = torch.vstack(embedding_parts).detach().cpu().numpy()
        else:
            # detach, return as numpy array
            embedding = torch.vstack(embedding_parts).detach().numpy()

        if np.isnan(embedding).any():
            raise RuntimeError(f"NaN detected in embeddings.")

        return embedding

In [ ]:
import torch

# 加载 .pth 文件
file_path = "/root/autodl-tmp/model.pth"
model_data = torch.load(file_path, map_location=torch.device('cpu'))

# 查看内容
print(model_data)

In [ ]:
decoder = model.decoder  
torch.save(decoder.state_dict(), 'decoder.ckpt')

In [ ]:
model

In [ ]:
model = torch.load('model.pth', weights_only=False)

In [ ]:
decoder = model.decoder  
torch.save(decoder.state_dict(), 'decoder.ckpt')

In [ ]:
encoder = model.encoder  
torch.save(encoder.state_dict(), 'encoder.ckpt')

In [ ]:
import json

# 假设你在训练模型时定义了超参数
hyperparameters = {
    "latent_dim": 128,  # 隐层维度
    "hidden_dim": [1024, 1024, 1024],  # 隐藏层的大小
    "dropout": 0.5,  # Dropout 概率
    "input_dropout": 0.4,  # 输入层的 Dropout 概率
    "margin": 0.05,  # 边际值
    "alpha": 0.001,  # 学习率相关的超参数
    "negative_selection": "semihard",  # 负选择方法
    "sample_across_studies": True,  # 是否跨研究进行采样
    "perturb_labels": True,  # 是否扰动标签
    "perturb_labels_fraction": 0.5,  # 扰动标签的比例
    "lr": 0.005,  # 学习率
    "l1_lambda": 0.0001,  # L1 正则化系数
    "l2_lambda": 0.01,  # L2 正则化系数
    "batch_size": 1000,  # 批次大小
    "iterations": 50000,  # 迭代次数
    "residual": False  # 残差是否使用
}

# 保存到超参数 JSON 文件
with open('hyperparameter.json', 'w') as f:
    json.dump(hyperparameters, f, indent=4)

In [ ]:
model.hparams

In [ ]:
# 保存到超参数 JSON 文件
with open('hyperparameter.json', 'w') as f:
    json.dump(model.hparams, f, indent=4)

In [92]:
class CellSearchKNN(CellEmbedding):
    """A class for searching similar cells using cell embeddings kNN."""

    def __init__(
        self,
        model_path: str,
        use_gpu: bool = False,
    ):
        """Constructor.

        Parameters
        ----------
        model_path: str
            Path to the directory containing model files.
        use_gpu: bool, default: False
            Use GPU instead of CPU.

        Examples
        --------
        >>> cs = CellSearchKNN(model_path="/opt/data/model")
        """

        super().__init__(
            model_path=model_path,
            use_gpu=use_gpu,
        )

        self.knn = None
        self.safelist = None
        self.blocklist = None

    def load_knn_index(self, knn_file: str):
        """Load the kNN index file

        Parameters
        ----------
        knn_file: str
            Filename of the kNN index.
        """

        import hnswlib
        import os

        if os.path.isfile(knn_file):
            self.knn = hnswlib.Index(space="cosine", dim=self.model.latent_dim)
            self.knn.load_index(knn_file)
        else:
            print(f"Warning: No KNN index found at {knn_file}")
            self.knn = None


    def get_nearest_neighbors(
        self, embeddings: "numpy.ndarray", k: int = 50, ef: int = 100
    ) -> Tuple["numpy.ndarray", "numpy.ndarray"]:
        """Get nearest neighbors.
        Used by classes that inherit from CellEmbedding and have an instantiated kNN.

        Parameters
        ----------
        embeddings: numpy.ndarray
            Embeddings as a numpy array.
        k: int, default: 50
            The number of nearest neighbors.
        ef: int, default: 100
            The size of the dynamic list for the nearest neighbors.
            See https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md

        Returns
        -------
        nn_idxs: numpy.ndarray
            A 2D numpy array of nearest neighbor indices [num_cells x k].
        nn_dists: numpy.ndarray
            A 2D numpy array of nearest neighbor distances [num_cells x k].

        Examples
        --------
        >>> nn_idxs, nn_dists = get_nearest_neighbors(embeddings)
        """

        if self.knn is None:
            raise RuntimeError("kNN is not initialized.")
        self.knn.set_ef(ef)
        return self.knn.knn_query(embeddings, k=k)